In [1]:
#VGG
import d2lzh as d2l
from mxnet import gluon ,init,nd
from mxnet.gluon import nn

def vgg_block(num_convs,num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels,kernel_size=3,padding=1,activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2,strides=2))
    return blk


In [2]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    net = nn.Sequential()
    #卷积层
    for (num_convs,num_channels) in conv_arch:
        net.add(vgg_block(num_convs,num_channels))
    #全连接层
    net.add(nn.Dense(512,activation='relu'),nn.Dropout(0.5),
            nn.Dense(128,activation='relu'),nn.Dropout(0.5),
            nn.Dense(10)
           )
    return net

net =vgg(conv_arch)

In [3]:
net.initialize()
X = nd.random.uniform(shape=(1,1,224,224))

for blk in net:
    X = blk(X)
    print(blk.name,'output shape:\t',X.shape)

sequential1 output shape:	 (1, 64, 112, 112)
sequential2 output shape:	 (1, 128, 56, 56)
sequential3 output shape:	 (1, 256, 28, 28)
sequential4 output shape:	 (1, 512, 14, 14)
sequential5 output shape:	 (1, 512, 7, 7)
dense0 output shape:	 (1, 512)
dropout0 output shape:	 (1, 512)
dense1 output shape:	 (1, 128)
dropout1 output shape:	 (1, 128)
dense2 output shape:	 (1, 10)


In [6]:
ratio = 4
small_conv_arch = [(pair[0],pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)


In [7]:
lr, num_epochs, batch_size, ctx = 0.05, 5, 128, d2l.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
num_epochs)

training on gpu(0)


MXNetError: [23:43:34] c:\jenkins\workspace\mxnet-tag\mxnet\3rdparty\mshadow\mshadow\./cuda/tensor_gpu-inl.cuh:110: Check failed: err == cudaSuccess (2 vs. 0) : Name: MapPlanKernel ErrStr:out of memory